# Building QA system with LLM experts in Law, Medicine and Finance

Intro: We build a QA system managed by the manager agent. Manager agent can obtain questions from human input and assign the question to three different LLMs regarding their experts including Law, Medicine and Finance

Pre-step: Host the AdaptLLM/medicine-chat, AdaptLLM/law-chat and AdaptLLM/finance-chat models using fastchat via openaiapi

In [11]:
# defining the educator class for chat
from typing import List
from agentlite.agents import ABCAgent, BaseAgent
from agentlite.llm.agent_llms import BaseLLM, get_llm_backend
from agentlite.llm.LLMConfig import LLMConfig
from agentlite.logging.multi_agent_log import AgentLogger
from agentlite.actions.BaseAction import BaseAction
from env import openai_api_key
from agentlite.commons import AgentAct, TaskPackage
from agentlite.actions import ThinkAct, FinishAct
from agentlite.actions.InnerActions import INNER_ACT_KEY
from agentlite.agents.agent_utils import AGENT_CALL_ARG_KEY
import openai

## Define and initialize the agent

In [12]:
# Define the Domain Expert agent
class DomainExpert(ABCAgent):
    def __init__(self, domain):
        name = "Expert_" + domain
        role = f"""You are an expert in {domain}, and you can answer questions in this domain given by user."""
        self.domain = domain
        self.temperature = 0.5
        self.stop = ['\n']
        super().__init__(
            name=name,
            role=role,
        ) 
    
    def __call__(self, question: TaskPackage):
        openai.api_key = "EMPTY"  # Not support yet
        openai.base_url = "http://localhost:8000/v1/"
        prompt = question.instruction + " " + "[/INST]"

        completion = openai.completions.create(
                    model=self.domain,
                    temperature=self.temperature,
                    stop=self.stop,
                    prompt=prompt,
                    max_tokens=128, #self.max_tokens,
                )
    
        output = completion.choices[0].text
        return output

# Initialize logger
agent_logger = AgentLogger(PROMPT_DEBUG_FLAG=False)

law_agent = DomainExpert("law")
finance_agent = DomainExpert("finance")
medicine_agent = DomainExpert("medicine")



In [13]:
# Define the human agent
class Human(ABCAgent):
    def __init__(self):
        name = "Human"
        role = """You are human to give questions to all the other agents. You can ask questions for law, medicine and finance."""
        super().__init__(
            name=name,
            role=role,
        ) 
    
    def __call__(self, question):
        print(question)
        print("Please input your instruction here: ")
        instruction = input()

        return f"My instruction is: {instruction}"
    
# Initialize human agent
human_agent = Human()

In [14]:
# define manager agent
from agentlite.agents import ManagerAgent

llm_config_dict = {"llm_name": "gpt-4-32k", "temperature": 0.0, "openai_api_key": openai_api_key}
llm_config = LLMConfig(llm_config_dict)
llm = get_llm_backend(llm_config)


# Initialize manager agent
manager_agent_info = {
    "name": "manager_agent",
    "role": "You are controlling the medicine_agent, finance_agent, law_agent and the human_agent. Each time you need to acquire a question from the human_agent, and you should decide which agent to answer the question."}
team = [medicine_agent, finance_agent, law_agent, human_agent]
manager_agent = ManagerAgent(name=manager_agent_info["name"], role=manager_agent_info["role"], llm=llm, TeamAgents=team,logger=agent_logger)

## Add an illustration example for the manager agent

In [15]:
# Add illustration example for the manager agent
exp_task = "Follow instructions from human to ask questions on different domains"
exp_task_pack = TaskPackage(instruction=exp_task)


act_1 = AgentAct(
    name=ThinkAct.action_name,
    params={INNER_ACT_KEY: f"""I need to get a question from human first."""
    },
)
obs_1 = "OK"


act_2 = AgentAct(
    name=human_agent.name,
    params={AGENT_CALL_ARG_KEY: "Do you have any questions regarding law, finance and medicine?",
        },
)
obs_2 = """What medicine should I take for a headache?"""


act_3 = AgentAct(
    name=ThinkAct.action_name,
    params={INNER_ACT_KEY: f"""It is a question regarding medicine. I need to ask the medicine_agent to answer the question."""
    },
)
obs_3 = "OK"


act_4 = AgentAct(
    name=medicine_agent.name,
    params={AGENT_CALL_ARG_KEY: "What medicine should the user take for a headache?",
        },
)
obs_4 = """Aspirin: Aspirin is another NSAID that can help relieve headache pain. It works by reducing inflammation and blocking pain signals in the brain."""


act_5 = AgentAct(
    name=ThinkAct.action_name,
    params={INNER_ACT_KEY: f"""I have answered the question from human. I need to collect the next question from human."""
    },
)
obs_5 = "OK"


act_6 = AgentAct(
    name=human_agent.name,
    params={AGENT_CALL_ARG_KEY: "Do you have another questions regarding law, finance and medicine?",
        },
)
obs_6 = """Hong long do I have to sit in jail for hit and run?"""

act_7 = AgentAct(
    name=ThinkAct.action_name,
    params={INNER_ACT_KEY: f"""It is a question regarding law. I need to ask the law_agent to answer the question."""
    },
)
obs_7 = "OK"


act_8 = AgentAct(
    name=law_agent.name,
    params={AGENT_CALL_ARG_KEY: "How long does a person have to sit in jail for hit and run?",
        },
)
obs_8 = """The legal penalties include up to 6 months in the country jail."""


act_9 = AgentAct(
    name=ThinkAct.action_name,
    params={INNER_ACT_KEY: f"""I have answered the question from human. I need to collect the next question from human."""
    },
)
obs_9 = "OK"


act_10 = AgentAct(
    name=human_agent.name,
    params={AGENT_CALL_ARG_KEY: "Do you have another questions regarding law, finance and medicine?",
        },
)
obs_10 = """What factors should investors consider when evaluating the risk of an investment?"""

act_11 = AgentAct(
    name=ThinkAct.action_name,
    params={INNER_ACT_KEY: f"""It is a question regarding finance. I need to ask the finance_agent to answer the question."""
    },
)
obs_11 = "OK"


act_12 = AgentAct(
    name=finance_agent.name,
    params={AGENT_CALL_ARG_KEY: "What factors should investors consider when evaluating the risk of an investment?",
        },
)
obs_12 = """Market rish, inflation risk, interest rate risk, liquidity risk, business risk, and political risk."""



act_13 = AgentAct(
    name=ThinkAct.action_name,
    params={INNER_ACT_KEY: f"""I have answered the question from human. I need to collect the next question from human."""
    },
)
obs_13 = "OK"


act_14 = AgentAct(
    name=human_agent.name,
    params={AGENT_CALL_ARG_KEY: "Do you have another questions regarding law, finance and medicine?",
        },
)
obs_14 = """We can end the session now."""


act_15 = AgentAct(name=FinishAct.action_name, params={INNER_ACT_KEY: "I have no more instructions from human. End the session."})
obs_15 = "Session ended."

exp_act_obs = [(act_1, obs_1), (act_2, obs_2), (act_3, obs_3), (act_4, obs_4), (act_5, obs_5), (act_6, obs_6), (act_7, obs_7), (act_8, obs_8), (act_9, obs_9), (act_10, obs_10), (act_11, obs_11), (act_12, obs_12), (act_13, obs_13), (act_14, obs_14), (act_15, obs_15)]
manager_agent.add_example(
    task = exp_task_pack, action_chain = exp_act_obs
)

## Begin asking questions !

In [16]:

exp_task = "Follow instructions from human to ask questions."
exp_task_pack = TaskPackage(instruction=exp_task)

manager_agent(exp_task_pack)

Agent manager_agent receives the following TaskPackage:
[
	Task ID: be3220e7-ce7b-49bd-afd4-6f8219f52d03
	Instruction: Follow instructions from human to ask questions.
]
====manager_agent starts execution on TaskPackage be3220e7-ce7b-49bd-afd4-6f8219f52d03====


Agent manager_agent takes 0-step Action:
{
	name: Think
	params: {'response': 'I need to get a question from human first.'}
}
Observation: OK
Agent manager_agent takes 1-step Action:
{
	name: Human
	params: {'Task': 'Do you have any questions regarding law, finance and medicine?'}
}
Task ID: be3220e7-ce7b-49bd-afd4-6f8219f52d03
Instruction: Do you have any questions regarding law, finance and medicine?
Task Creator: 
Task Completion:active
Answer: 
Task Executor: 
Please input your instruction here: 
Observation: My instruction is: What question can I ask?
Agent manager_agent takes 2-step Action:
{
	name: Think
	params: {'response': 'This is not a specific question in the domains of law, finance, or medicine. I need to ask the human to provide a specific question.'}
}
Observation: OK
Agent manager_agent takes 3-step Action:
{
	name: Human
	params: {'Task': 'Could you please ask a specific question in the domains of law, finance, or medicine?'}
}
Task ID: be3220e7-ce7b-49bd-afd4-6f8219f

'I have no more instructions from human. End the session.'